In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:,.0f}'.format

In [2]:
# 분류 테이블
claf_pnc = pd.read_excel('C:/FSS_FinesReport/Setting/RBC분류.xlsx', sheet_name='일반')
claf_auto = pd.read_excel('C:/FSS_FinesReport/Setting/RBC분류.xlsx', sheet_name='자동차')

In [3]:
def genDates(date):
    # 필요한 업무보고서 일자 생성
    date = str(date)
    if date[2:] == '12':
        res = [date, str(int(date) - 100), str(int(date) - 200)]
    else:
        res = [date, str(int(date) - 100)[:2] + '12', str(int(date) - 100), str(int(date) - 200)[:2] + '12', str(int(date) - 200), str(int(date) - 300)[:2] + '12', str(int(date) - 300)]
    return res

In [22]:
def fss_rpt(num, date):
    # 업무보고서와 기준일자를 받아서 테이블을 출력하는 함수
    num = str(num)
    while len(num) < 3: num = '0' + num
    rpt = pd.read_excel('C:/FSS_FinesReport/Setting/업무보고서_'+str(date)+'.xlsx', sheet_name='AI'+num)
    
    if num == '059':
        rpt = rpt.iloc[11:,1:]
        rpt.index = pd.RangeIndex(start=14,stop=155, step=1)
        rpt.columns = pd.RangeIndex(start=2,stop=80, step=1)
        rpt.iloc[:,1:] = rpt.iloc[:,1:].astype(float, inplace=True).fillna(0)
    elif num == '060':
        rpt = pd.read_excel('C:/FSS_FinesReport/Setting/업무보고서_'+str(1712)+'.xlsx', sheet_name='AI060')
        rpt = rpt.iloc[11:]
        rpt.index = pd.RangeIndex(start=14,stop=146, step=1)
        rpt.columns = pd.RangeIndex(start=1,stop=60, step=1)
        rpt.iloc[:,1:] = rpt.iloc[:,1:].astype(float, inplace=True).fillna(0)
    else:
        print('AI{:}_{:}는 목록에 없습니다'.format(num,date))
        return None
    
    print('[AI{:}_{:}]가 생성 되었습니다.'.format(num,date))
    return rpt

In [34]:
def fss_filter(num, date):
    # 사업실적표를 받아서 필요한 열만 추출하는 함수
    rpt = fss_rpt(num, date)
    num = str(num)
    while len(num) < 3: num = '0' + num
    if num == '059':
        cols = ['RBC분류','모니터링분류','구분',7,8,12,13,17,18,23,32,45,52,55,67]
        colnames = ['RBC분류','모니터링분류','구분','원수보험료','원수해약환급금','수재보험료','수재해약환급금','출재보험료','출재해약환급금','경과보험료','원수지급준비금','수재지급준비금','출재지급준비금','발생손해액','순사업비']
        filtered = rpt.merge(claf_pnc, left_on=2, right_on='구분', how='inner').drop(2, axis=1)[cols]
        filtered.columns = colnames
        filtered = filtered.dropna()
        filtered['원수보험료'] = filtered['원수보험료'] - filtered['원수해약환급금']
        filtered['수재보험료'] = filtered['수재보험료'] - filtered['수재해약환급금']
        filtered['출재보험료'] = filtered['출재보험료'] - filtered['출재해약환급금']
        filtered.drop(['원수해약환급금','수재해약환급금','출재해약환급금'], axis=1, inplace=True)
    if num == '060':
        cols = ['RBC분류','모니터링분류','구분',5,6,10,11,15,16,21,27,34,41,44,58]
        colnames = ['RBC분류','모니터링분류','구분','원수보험료','원수해약환급금','수재보험료','수재해약환급금','출재보험료','출재해약환급금','경과보험료','원수지급준비금','수재지급준비금','출재지급준비금','발생손해액','순사업비']
        filtered = rpt.merge(claf_auto, left_on=1, right_on='구분', how='inner').drop(2, axis=1)[cols]
        filtered.columns = colnames
        filtered = filtered.dropna()
        filtered['원수보험료'] = filtered['원수보험료'] - filtered['원수해약환급금']
        filtered['수재보험료'] = filtered['수재보험료'] - filtered['수재해약환급금']
        filtered['출재보험료'] = filtered['출재보험료'] - filtered['출재해약환급금']
        filtered.drop(['원수해약환급금','수재해약환급금','출재해약환급금'], axis=1, inplace=True)    
    return filtered

In [49]:
def rbc_sum(filtered):
    # 정제된 사업실적표를 RBC분류 단위로 합치는 함수
    summed = filtered.drop(['모니터링분류','구분'], axis=1).groupby('RBC분류').sum()
    return summed

In [53]:
def rbc_expo_prem(summed):
    # 누적 수입보험료
    return summed[['원수보험료','수재보험료','출재보험료']]
def rbc_expo_rsrv(summed):
    # 지급준비금
    return summed[['원수지급준비금','수재지급준비금','출재지급준비금']]
def rbc_comb(summed):
    # 누적 합산비율
    return summed[['경과보험료','발생손해액','순사업비']]

In [71]:
def rbc_expo_prem_1yr(num, date):
    # 직전 1년 수입보험료
    dates = genDates(date)
    if str(date)[2:] == '12':
        prem_expo = rbc_expo_prem(rbc_sum(fss_filter(num,dates[0])))
    else:
        prem_expo = rbc_expo_prem(rbc_sum(fss_filter(num,dates[0]))) + rbc_expo_prem(rbc_sum(fss_filter(num,dates[1]))) - rbc_expo_prem(rbc_sum(fss_filter(num,dates[2])))
    return prem_expo

def rbc_comb_3yrs(num, date):
    # 직전 3년 합산비율
    dates = genDates(date)
    if str(date)[2:] == '12':
        comb_0 = rbc_comb(rbc_sum(fss_filter(num,dates[0])))
        comb_1 = rbc_comb(rbc_sum(fss_filter(num,dates[1])))
        comb_2 = rbc_comb(rbc_sum(fss_filter(num,dates[2])))
    else:
        comb_0 = rbc_comb(rbc_sum(fss_filter(num,dates[0]))) + rbc_comb(rbc_sum(fss_filter(num,dates[1]))) - rbc_comb(rbc_sum(fss_filter(num,dates[2])))
        comb_1 = rbc_comb(rbc_sum(fss_filter(num,dates[2]))) + rbc_comb(rbc_sum(fss_filter(num,dates[3]))) - rbc_comb(rbc_sum(fss_filter(num,dates[4])))
        comb_2 = rbc_comb(rbc_sum(fss_filter(num,dates[4]))) + rbc_comb(rbc_sum(fss_filter(num,dates[5]))) - rbc_comb(rbc_sum(fss_filter(num,dates[6])))
            
    comb_0.columns = comb_0.columns + '_0년'
    comb_1.columns = comb_1.columns + '_1년'
    comb_2.columns = comb_2.columns + '_2년'
    comb = pd.concat([comb_0, comb_1, comb_2], axis=1)
    return comb

In [73]:
def rbc_result(num, date):
    # RBC 입력데이터
    return pd.concat([rbc_expo_prem_1yr(num, date), rbc_expo_rsrv(rbc_sum(fss_filter(num, date))), rbc_comb_3yrs(num, date)], axis=1)